# Emotion Prediction


It uses both strategy (1) test set 1: consider all samples corresponding to opportune moment, (2) test set 2: consider all samples regardless of opportune or non-opportune. 

In [ ]:
# import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from scipy.stats import zscore
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel

In [ ]:
# Load physiological data
def load_physiological_data(file_path):
    columns = ["time", "ECG", "BVP", "GSR", "Resp", "Skin_Temp", "EMG_z", "EMG_c", "EMG_t"]
    df = pd.read_csv(file_path, sep="\t", names=columns)
    return df

# Load valence-arousal data
def load_valence_arousal_data(file_path):
    df = pd.read_csv(file_path, sep="\t", names=["jsttime", "valence", "arousal"])
    return df

In [3]:
# Downsample data to 1Hz and create 5-second windows
def preprocess_data(physio_df, va_df, window_size=5):
    # Downsample physiological data from 1000Hz to 1Hz
    physio_downsampled = physio_df.iloc[::1000].copy()  # Simple decimation
    
    # Downsample valence-arousal data from 20Hz to 1Hz
    va_downsampled = va_df.iloc[::20].copy()
    
    # Create windows for both datasets
    start_time = min(physio_downsampled["time"].min(), va_downsampled["jsttime"].min())
    
    physio_downsampled["window"] = ((physio_downsampled["time"] - start_time) // window_size).astype(int)
    va_downsampled["window"] = ((va_downsampled["jsttime"] - start_time) // window_size).astype(int)
    
    # Aggregate by window
    physio_segmented = physio_downsampled.groupby("window").mean().reset_index()
    va_segmented = va_downsampled.groupby("window").mean().reset_index()
    
    return physio_segmented, va_segmented

In [4]:
# RuLSIF Algorithm for Change-Point Detection
def compute_rulsif_change_scores(X, alpha=0.1, sigma=0.1, lambda_param=1e-3):
    n = len(X) - 1
    change_scores = np.zeros(n)
    
    for i in range(n):
        X_t, X_t_next = X[i], X[i + 1]
        
        # Compute Gaussian Kernel Matrix
        K_t = rbf_kernel(X_t.reshape(-1, 1), X_t.reshape(-1, 1), gamma=1/(2*sigma**2))
        K_t_next = rbf_kernel(X_t_next.reshape(-1, 1), X_t_next.reshape(-1, 1), gamma=1/(2*sigma**2))
        
        # Compute Weights using Least Squares Importance Fitting (LSIF)
        H = alpha * K_t + (1 - alpha) * K_t_next + lambda_param * np.eye(K_t.shape[0])
        h = np.mean(K_t, axis=1)
        
        theta = np.linalg.solve(H, h)
        
        # Compute Change Score
        change_scores[i] = np.mean(np.square(K_t_next.dot(theta) - 1))
    
    return change_scores

In [5]:
# Identify significant changes and label opportune moments
def label_opportune_moments(change_scores):
    mean, std = np.mean(change_scores), np.std(change_scores)
    threshold = mean + 3 * std
    outliers = change_scores > threshold
    
    # Clustering the remaining scores
    valid_indices = np.where(~outliers)[0]  # Indices of non-outliers
    valid_scores = change_scores[valid_indices]
    
    if len(valid_scores) > 1:  # Ensure there are enough points for clustering
        kmeans = KMeans(n_clusters=2, random_state=42).fit(valid_scores.reshape(-1, 1))
        centroids = kmeans.cluster_centers_.flatten()
        high_cluster = np.argmax(centroids)
        high_values = (kmeans.labels_ == high_cluster) & (valid_scores > centroids[high_cluster])
        
        # Map high_values back to the original indices
        high_values_original = np.zeros_like(change_scores, dtype=bool)
        high_values_original[valid_indices] = high_values
    else:
        # If there are not enough valid scores, treat all as non-opportune
        high_values_original = np.zeros_like(change_scores, dtype=bool)
    
    # Mark opportune moments
    opportune_moments = np.where(outliers | high_values_original)[0]
    return opportune_moments

In [6]:
# Align valence-arousal data with physiological data windows
def align_data(physiological_data, valence_arousal_data, window_size=5):
    # Convert jsttime to window index
    start_time = physiological_data["time"].min()
    valence_arousal_data["window"] = ((valence_arousal_data["jsttime"] - start_time) // window_size).astype(int)
    
    # Merge data on window index
    merged_data = pd.merge(physiological_data, valence_arousal_data, on="window", how="inner")
    return merged_data

In [7]:
# Map valence-arousal to emotion classes
def map_to_emotion_classes(valence, arousal):
    # Define emotion classes based on valence and arousal
    if valence >= 0 and arousal >= 0:
        return "Happy"
    elif valence >= 0 and arousal < 0:
        return "Relaxed"
    elif valence < 0 and arousal >= 0:
        return "Stressed"
    else:
        return "Sad"

In [8]:
# Train and evaluate classification model
def train_and_evaluate_classifier(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = model.predict(X_test)
    
    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    report = classification_report(y_test, y_pred)
    
    print("Classification Report:")
    print(report)
    
    return accuracy, f1

In [ ]:
# Main execution for all users
if __name__ == "__main__":
    # Initialize a list to store results for all users
    results = []
    
    # Define features for detecting opportune moments and for training the model
    opportune_features = [ "EMG_z","GSR","BVP" ]  # Features used for detecting opportune moments
    model_features = ["ECG", "EMG_c", "EMG_t","Resp", "Skin_Temp"]  # Features used for training the model
    
    # Loop through all 30 users
    for user_id in range(1, 31):  # Assuming user IDs are from 1 to 30
        try:
           # Load data
            physio_data = load_physiological_data(f"sub{user_id}_DAQ.txt")
            va_data = load_valence_arousal_data(f"sub{user_id}_joystick.txt")
            
          # Preprocess and align data
            physio_segmented, va_segmented = preprocess_data(physio_data, va_data)
            merged_data = pd.merge(physio_segmented, va_segmented, on="window", how="inner")
            
            # Map valence-arousal to emotion classes
            merged_data["emotion"] = merged_data.apply(
                lambda row: map_to_emotion_classes(row["valence"], row["arousal"]), 
                axis=1
            )
            
            # Prepare features (X) and targets (y)
            X_opportune = merged_data[opportune_features].apply(zscore).values
            X_model = merged_data[model_features]
            y = merged_data["emotion"]
            
            # Split data into train and test sets (baseline model)
            X_train, X_test, y_train, y_test = train_test_split(
                X_model, y, test_size=0.2, random_state=42
            )
            
            # Train and evaluate baseline model - with all features at all moments
            accuracy_all, f1_all = train_and_evaluate_classifier(
                X_train, X_test, y_train, y_test
            )
            
            # Detect opportune moments
            change_scores = compute_rulsif_change_scores(X_opportune)
            opportune_moments = label_opportune_moments(change_scores)
            
            # Filter data for opportune moments
            opportune_data = merged_data[merged_data["window"].isin(opportune_moments)]
            
            # Skip user if no opportune moments found
            if len(opportune_data) == 0:
                print(f"No opportune moments found for User {user_id}. Skipping...")
                continue
                
            # Prepare features for opportune moments
            X_opportune_train = opportune_data[model_features]
            y_opportune = opportune_data["emotion"]
            
            # Check if we have enough samples for splitting
            MIN_SAMPLES = 5  # Minimum samples required
            if len(X_opportune_train) < MIN_SAMPLES:
                print(f"Only {len(X_opportune_train)} opportune moments for User {user_id}. Skipping...")
                continue
                
            try:
                # Split data into train and test sets for opportune moments
                X_train_opportune, X_test_opportune, y_train_opportune, y_test_opportune = train_test_split(
                    X_opportune_train, y_opportune, test_size=0.2, random_state=42
                )
                
                # Train and evaluate opportune moments model
                accuracy_opportune, f1_opportune = train_and_evaluate_classifier(
                    X_train_opportune, X_test_opportune, y_train_opportune, y_test_opportune
                )
                
            except Exception as e:
                print(f"Error processing User {user_id}: {str(e)}. Skipping...")
                continue
                
            # Store results for the current user
            results.append({
                "User ID": user_id,
                "Accuracy (All)": accuracy_all,
                "F1 Score (All)": f1_all,
                "Accuracy (Opportune)": accuracy_opportune,
                "F1 Score (Opportune)": f1_opportune,
                "Opportune Moments Count": len(opportune_moments)
            })
            
            print(f"Successfully processed User {user_id}")
            
        except FileNotFoundError:
            print(f"Data files for User {user_id} not found. Skipping...")
            continue
        except Exception as e:
            print(f"Unexpected error processing User {user_id}: {str(e)}. Skipping...")
            continue
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    
    # Save results to a CSV file
    results_df.to_csv("emotion_classification_results_robust.csv", index=False)
    
    # Print the results table
    print("\nFinal Results:")
    print(results_df)

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

       Happy       0.84      0.97      0.90        73
     Relaxed       0.67      0.29      0.40        14
         Sad       0.00      0.00      0.00         1
    Stressed       0.88      0.64      0.74        11

    accuracy                           0.83        99
   macro avg       0.59      0.47      0.51        99
weighted avg       0.81      0.83      0.80        99

Classification Report:
              precision    recall  f1-score   support

       Happy       0.82      0.91      0.86        34
     Relaxed       1.00      0.25      0.40         4
    Stressed       0.73      0.67      0.70        12

    accuracy                           0.80        50
   macro avg       0.85      0.61      0.65        50
weighted avg       0.81      0.80      0.78        50

Successfully processed User 1


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

       Happy       0.90      0.93      0.92        58
     Relaxed       0.82      0.82      0.82        22
         Sad       0.00      0.00      0.00         1
    Stressed       0.88      0.83      0.86        18

    accuracy                           0.88        99
   macro avg       0.65      0.65      0.65        99
weighted avg       0.87      0.88      0.87        99



C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Classification Report:
              precision    recall  f1-score   support

       Happy       0.70      1.00      0.82        30
     Relaxed       1.00      1.00      1.00         2
         Sad       0.00      0.00      0.00         5
    Stressed       0.67      0.29      0.40        14

    accuracy                           0.71        51
   macro avg       0.59      0.57      0.56        51
weighted avg       0.63      0.71      0.63        51

Successfully processed User 26
Classification Report:
              precision    recall  f1-score   support

       Happy       0.92      0.91      0.91        64
     Relaxed       0.73      0.76      0.74        21
         Sad       1.00      1.00      1.00         2
    Stressed       1.00      1.00      1.00        12

    accuracy                           0.89        99
   macro avg       0.91      0.92      0.91        99
weighted avg       0.89      0.89      0.89        99



C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

       Happy       0.91      0.97      0.94        32
     Relaxed       0.80      0.57      0.67         7
         Sad       0.00      0.00      0.00         1
    Stressed       0.89      1.00      0.94         8

    accuracy                           0.90        48
   macro avg       0.65      0.64      0.64        48
weighted avg       0.87      0.90      0.88        48

Successfully processed User 27
Classification Report:
              precision    recall  f1-score   support

       Happy       0.83      0.95      0.89        65
     Relaxed       1.00      0.29      0.44         7
         Sad       0.88      0.88      0.88        16
    Stressed       0.83      0.45      0.59        11

    accuracy                           0.84        99
   macro avg       0.88      0.64      0.70        99
weighted avg       0.85      0.84      0.82        99



C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

       Happy       0.77      1.00      0.87        30
     Relaxed       0.00      0.00      0.00         3
         Sad       1.00      0.75      0.86        12
    Stressed       1.00      0.62      0.77         8

    accuracy                           0.83        53
   macro avg       0.69      0.59      0.62        53
weighted avg       0.81      0.83      0.80        53

Successfully processed User 28
Classification Report:
              precision    recall  f1-score   support

       Happy       0.89      0.99      0.94        85
     Relaxed       1.00      0.33      0.50         9
    Stressed       0.50      0.20      0.29         5

    accuracy                           0.89        99
   macro avg       0.80      0.51      0.57        99
weighted avg       0.88      0.89      0.87        99

Classification Report:
              precision    recall  f1-score   support

       Happy       0.91      